# Download and process DEM into a single geotiff at native resolution

In [61]:
import os
import sys
import shutil
import tarfile
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling                                                                                        

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [38]:
# Get the required info from the config file
data_path            = cs.read_from_config(config_file,'data_path')
geospatial_temp_path = cs.read_from_config(config_file,'geospatial_temp_path')
dem_path             = cs.read_from_config(config_file,'dem_path')
merit_url            = cs.read_from_config(config_file,'dem_url')
download_area        = cs.read_from_config(config_file,'geospatial_area')

### Convert geospatial coordinates to specific download coordinates for this data product

In [27]:
download_coordinates = geospatial_coordinates_to_download_coordinates(download_area, 'merit')

### Download data

In [29]:
usr,pwd = cs.read_merit_credentials()

In [30]:
merit_lon,merit_lat = cs.convert_coordinates_to_merit_download_lists(download_coordinates)

In [34]:
download_folder = Path(data_path) / geospatial_temp_path / 'merit' / 'download'

In [36]:
download_folder.mkdir(parents=True, exist_ok=True)

In [39]:
for dl_lon in merit_lon:
    for dl_lat in merit_lat:
        file_url = merit_url.format(dl_lat,dl_lon) # Replace placeholder values in url with download coordinates
        cs.download_merit_hydro_grid(file_url, usr, pwd, download_folder)

Successfully downloaded http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/distribute/v1.0/elv_n00w180.tar
Successfully downloaded http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/distribute/v1.0/elv_n30w180.tar
Successfully downloaded http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/distribute/v1.0/elv_n60w180.tar
Successfully downloaded http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/distribute/v1.0/elv_n30w150.tar
Successfully downloaded http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/distribute/v1.0/elv_n60w150.tar
Successfully downloaded http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/distribute/v1.0/elv_n00w120.tar
Successfully downloaded http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/distribute/v1.0/elv_n30w120.tar
Successfully downloaded http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/distribute/v1.0/elv_n60w120.tar
Successfully downloaded http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/distribute/v1.0/elv_n00w090.tar
Successfully downloaded http

### Unzip

In [41]:
# Find the file names
files = [ f for f in os.listdir(download_folder) if os.path.isfile(
            os.path.join(download_folder,f))]

In [43]:
for file in files:
    tar_file = tarfile.open(download_folder/file)
    tar_file.extractall(download_folder)
    tar_file.close()
    os.remove(download_folder/file)

### Merge

In [44]:
# Define the output file
merged_file = 'merit_hydro_elv.tif'

In [45]:
# Find the file names
all_files = []
for dir_path, dir_names, file_names in os.walk(download_folder):
    for file_name in file_names:
        if file_name.endswith('.tif'): # ensure we don't accidentally get .aux files from QGIS or something similar
            all_files.append(os.path.join(dir_path,file_name))

In [47]:
merged_folder = Path(data_path) / geospatial_temp_path / 'merit' / 'raw'
merged_folder.mkdir(parents=True, exist_ok=True)

In [54]:
# Convert subsetting area into a usable GDAL setting
# subset_area = [lon_min, lon_max, lat_min, lat_max]
# GDAL window = [ulx, uly, lrx, lry]; [upper left x, upper left y, lower right x, lower right y]
# Mapping:
#   ulx = lon_min = subset_area[0]
#   uly = lat_max = subset_area[3]
#   lrx = lon_max = subset_area[1]
#   lry = lat_min = subset_area[2]
subset_coor = download_area.split(',')
window = [subset_coor[0], subset_coor[3], subset_coor[1], subset_coor[2]]

In [56]:
cs.merge_merit_downloads_into_area_of_interest(all_files, str(merged_folder/merged_file), window)

### Delete the individual GeoTIFF files to save space

In [63]:
if os.path.isdir(download_folder): 
    shutil.rmtree(download_folder)